# Model 3 classical loudspeakers

Usefull for debugging 

References: 
- Mäkivirta, Aki & Liski, Juho & Välimäki, Vesa. (2018). Modeling and Delay-Equalizing Loudspeaker Responses. Journal of the Audio Engineering Society. 66. 922-934. 10.17743/jaes.2018.0053. 
- Linkwitz, Siegfried H. (February 1976). "AES E-Library: Browse Entire Database". JAES Volume 24 Issue 1. The Audio Engineering Society: 2–8. 

In [ ]:
import math
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
freq = np.logspace(1+math.log10(2), 4+math.log10(2), 200)

In [ ]:
import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build

In [ ]:
ls = Biquad(1, 20, 48000, 10, 10)
hs = Biquad(0, 20000, 48000, 10, 5)
filter_one_way = [(-1, ls), (-1, hs)]

In [ ]:
one_way = peq_build(freq, filter_one_way)

In [ ]:
alt.Chart(pd.DataFrame({'freq': freq, 'dB': one_way})).mark_line().encode(
    x=alt.X('freq:Q', scale=alt.Scale(type="log", domain=[20, 20000], nice=False)), 
    y=alt.Y('dB:Q', scale=alt.Scale(domain=[-140, 10]))
)

In [ ]:
ls1 = Biquad(1,   80, 48000, 0.5, 10)
hs1 = Biquad(0, 5000, 48000, 0.5, 10)
ls2 = Biquad(1, 1000, 48000, 0.5, 10)
filter_woofer = [(1, ls1), (1, hs1)]
filter_tweeter = [(1, ls2)]

In [ ]:
woofer_two_way = peq_build(freq, filter_woofer)
tweeter_two_way = peq_build(freq, filter_tweeter)

In [ ]:
chart_woofer = alt.Chart(pd.DataFrame({'freq': freq, 'dB': woofer_two_way})).mark_line().encode(
    x=alt.X('freq:Q', scale=alt.Scale(type="log", domain=[20, 20000], nice=False)), 
    y=alt.Y('dB:Q', scale=alt.Scale(domain=[-40, 10])),
    color=alt.value('red')
)
chart_tweeter = alt.Chart(pd.DataFrame({'freq': freq, 'dB': tweeter_two_way})).mark_line().encode(
    x=alt.X('freq:Q', scale=alt.Scale(type="log", domain=[20, 20000], nice=False)), 
    y=alt.Y('dB:Q', scale=alt.Scale(domain=[-40, 10], zero=False, nice=False)),
    color=alt.value('green')
)
chart_woofer+chart_tweeter

In [ ]:
from spinorama.filter_peq import peq_butterworth_lowpass, peq_butterworth_highpass, peq_linkwitzriley_lowpass, peq_linkwitzriley_highpass

In [ ]:
bw2_lp = peq_build(freq, peq_butterworth_lowpass(2, 2000, 48000))
bw2_hp = peq_build(freq, peq_butterworth_highpass(2, 2000, 48000))
lr4_lp = peq_build(freq, peq_linkwitzriley_lowpass(4, 2000, 48000))
lr4_hp = peq_build(freq, peq_linkwitzriley_highpass(4, 2000, 48000))

In [ ]:
alt.Chart(
    pd.DataFrame({'freq': freq, 'LR4 lp': lr4_lp, 'LR4 hp': lr4_hp, 'BW2 lp': bw2_lp-10, 'BW2 hp': bw2_hp-10})
).transform_fold(['LR4 lp', 'LR4 hp', 'BW2 lp', 'BW2 hp']
).transform_filter('datum.value>-40'
).mark_line().encode(
    x=alt.X('freq:Q', scale=alt.Scale(type="log", domain=[20, 20000], nice=False)), 
    y=alt.Y('value:Q', scale=alt.Scale(domain=[-40, 10], nice=False)),
    color='key:N',
)